### Extraer polígonos de Polos del bienestar

Este ejercicio se hace para los polígonos de los polos del bienestar establecidos en el Plan México. Los polígonos se localizan en https://www.planmexico.gob.mx/polos

In [1]:
import pandas as pd
import requests
import os
import geopandas as gpd

In [2]:
url_basica="https://www.datosgeoespaciales.adyt.gob.mx/UPLOAD-MAPAS/mapa_planmexico/geodata/p_"
fin=".pmtiles"
#Crea directorio "capas" para descarga de archivos
if not os.path.exists("capas"):
    os.makedirs("capas")

In [3]:
#Descargar
for i in range(1,13):
    url=url_basica+str(i)+fin
    response = requests.get(url)
    with open("capas/p_"+str(i)+".pmtiles", "wb") as f:
        f.write(response.content)

In [4]:
#Sectores de cada polo para corrección respecto a archivos originales

sectores_dict = {
    "p_1": "semicon,electromovilidad,bienes",
    "p_2": "farma,petroquimica",
    "p_3": "agro,textil,electromovilidad,bienes,farma,petroquimica,aeroespacial",
    "p_4": "bienes",
    "p_5": "electromovilidad,bienes,semicon,aeroespacial",
    "p_6": "textil,agro,electromovilidad,bienes",
    "p_7": "textil,agro,electromovilidad,bienes,farma,semicon,petroquimica",
    "p_8": "textil,agro,farma,petroquimica,aeroespacial",
    "p_9": "textil,agro,electromovilidad,bienes,petroquimica,aeroespacial",
    "p_10": "agro,bienes,aeroespacial",
    "p_11": "agro,bienes",
    "p_12": "agro,aeroespacial"
}

In [6]:
# Carpeta con archivos
carpeta = "capas"
list_archivos = os.listdir(carpeta)
list_gdf = []

# Recorrer archivos
for archivo in list_archivos:
    if archivo.endswith(".pmtiles"):
        gdf = gpd.read_file(os.path.join(carpeta, archivo))

        # Quitar cve_sector si existe
        if 'cve_sector' in gdf.columns:
            gdf = gdf.drop(columns='cve_sector')

        # Determinar clave del archivo
        clave = archivo.split(".")[0]

        # Caso especial para p_10
        if clave == "p_10":
            columnas = ['geometry', 'mvt_id'] if 'mvt_id' in gdf.columns else ['geometry']
            gdf = gdf[columnas]
            gdf['corredor'] = "Golfo"
            gdf['id'] = 10
            gdf['nombre'] = "Golfo"

        # Agregar o corregir sector_estrategico si está en el diccionario
        if clave in sectores_dict:
            gdf['sector_estrategico'] = sectores_dict[clave]

        list_gdf.append(gdf)

# Concatenar todo
poligonos = pd.concat(list_gdf, ignore_index=True)
#Eliminar si la geografía es un punto
poligonos = poligonos[poligonos.geometry.type != 'Point']
#Cambiar proyección a EPSG:4326
poligonos = poligonos.set_crs(epsg=4326, allow_override=True)
print(poligonos.crs)

EPSG:4326


In [7]:
poligonos

,mvt_id,corredor,id,nombre,sector_estrategico,geometry
0,NaN,Franja fronteriza,1,Frontera Norte,"semicon,electromovilidad,bienes","MULTIPOLYGON (((-12584955.41276 3660741.26789,..."
1,NaN,Franja fronteriza,1,Frontera Norte,"semicon,electromovilidad,bienes","MULTIPOLYGON (((-12562578.47273 3657225.16459,..."
2,NaN,Franja fronteriza,1,Frontera Norte,"semicon,electromovilidad,bienes","MULTIPOLYGON (((-12562578.47273 3717897.05579,..."
3,NaN,Franja fronteriza,1,Frontera Norte,"semicon,electromovilidad,bienes","MULTIPOLYGON (((-12523442.71424 3735286.47973,..."
4,NaN,Franja fronteriza,1,Frontera Norte,"semicon,electromovilidad,bienes","MULTIPOLYGON (((-12562578.47273 3749771.29659,..."
...,...,...,...,...,...,...
63800,NaN,Centro,9,Centro,"textil,agro,electromovilidad,bienes,petroquimi...","MULTIPOLYGON (((-11350927.36423 2064774.33581,..."
63801,NaN,Centro,9,Centro,"textil,agro,electromovilidad,bienes,petroquimi...","MULTIPOLYGON (((-11323123.3952 2109308.45942, ..."
63802,NaN,Centro,9,Centro,"textil,agro,electromovilidad,bienes,petroquimi...","MULTIPOLYGON (((-11349245.74961 2064248.83124,..."
63803,NaN,Centro,9,Centro,"textil,agro,electromovilidad,bienes,petroquimi...","MULTIPOLYGON (((-11310234.2013 2074195.19955, ..."


In [10]:
#Salvar geojson
poligonos.to_file("poligonos_planmx.geojson", driver='GeoJSON')